# ALS 모델 기반 학습 및 평가 진행
- 메디스트림의 order 데이터 기반으로 아이템 추천을 진행합니다.
- 3개월치 데이터 중 도서 카테고리 아이템 추천을 진행합니다.
- train test 나눈 후 prediction 과 real item의 NDCG score를 통해 평가합니다.

# 추천 모델
- ALS MF, LMF, MP (총 3개)
- 총 3개의 추천을 진행하며 MF와 LMF 의 경우 콜드스타트 유저(신규 유저)인 경우 MP로 추천 진행

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import scipy.sparse as sparse
import random
import implicit
from implicit.als import AlternatingLeastSquares as ALS

%cd /home/user_3/medistream-recsys/Script
from preprocessing import drop_columns,dict_to_column,dict_to_set,set_to_column,key_to_element

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)

/home/user_3/medistream-recsys/Script


# 1.Dataload

In [6]:
# products name 확인 용
products_df = pd.read_json("/fastcampus-data/products/products.json")
products_df = key_to_element(['_id'],products_df)

100%|██████████| 5141/5141 [00:00<00:00, 465792.17it/s]


In [7]:
df = pd.read_json('/fastcampus-data/select_column_version_4.json')

In [8]:
from dateutil.relativedelta import relativedelta
from datetime import datetime

df['date_paid'] = pd.to_datetime(df['date_paid'])
# 3개월 전 날짜 확인
df['date_paid'].max()-relativedelta(months=6)

Timestamp('2022-03-13 08:59:21.151000+0000', tz='UTC')

In [9]:
date_state = "2022-03-13"

# paid orders만 가져오기
df['date_paid'] = pd.to_datetime(df['date_paid'])
df_only_paid = df[~df['date_paid'].isna()]
# 3개월치 데이터만 가져오기
df_date = df_only_paid[df_only_paid['date_paid'] >= date_state]
# 취소 안된 것만 가져오기
complete_df = df_date[(df_date['paid'] == True) & (df_date['cancelled']==False)]
# 도서 카테고리만 가져오기
only_book = complete_df[complete_df['name'] == '도서']

# 유저가 중복으로 아이템 구매 삭제
df_duplicated_book = only_book.drop_duplicates(subset=['customer_id','product_ids'])
df_book = df_duplicated_book.sort_values(by='date_paid').reset_index(drop=True)

# 도서, 소모품 카테고리
# df_book = complete_df[complete_df['name'].isin(['도서','소모품'])].sort_values(by='date_paid')

In [10]:
# none 값 확인하기
df_book.isna().sum()

_id                 0
date_created        0
regular_price       0
sale_price          0
three_months        0
date_paid           0
customer_id         0
paid                0
name_x              0
category_id_y       0
product_ids         0
quantity            0
price               0
price_total         0
age_group        1222
한의사 여부              2
사업자 여부              2
cancelled           0
name                0
slug                0
dtype: int64

## 전체 데이터 EDA

In [11]:
print('중복 제거 전:',len(only_book), '중복 제거 후:',len(df_book))

중복 제거 전: 8379 중복 제거 후: 8328


In [12]:
print('전체 데이터 수:',len(df_book))

전체 데이터 수: 8328


In [13]:
print('아이템 수:',len(df_book.product_ids.unique()),'유저 수:',len(df_book.customer_id.unique()))

아이템 수: 275 유저 수: 3251


# 2.train test split
- 마지막 3주 분량을 test로 선정합니다.
- train 없는 test 아이템을 삭제 진행합니다.

In [14]:
from datetime import datetime, timedelta
df_book['date_paid'].max()

Timestamp('2022-09-13 08:51:40+0000', tz='UTC')

In [15]:
datetime(2022,9,13)-timedelta(days=21)

datetime.datetime(2022, 8, 23, 0, 0)

In [16]:
date = '2022-08-23'
train_before_preprocess = df_book[df_book['date_paid'] < date]
test_before_preprocess = df_book[df_book['date_paid'] >= date]

## 전체 아이템 중복 확인

In [17]:
# product_ids, name_x 수는 일치
len(df_book.product_ids.unique()), len(df_book.name_x.unique())

(275, 275)

In [18]:
# 중복 제거 후 수 비교 확인
# 252로 일치하여 문제 없음
len(df_book.drop_duplicates(subset=['product_ids','name_x']).name_x.unique())

275

## train test 아이템 중복 확인

In [19]:
len(train_before_preprocess.product_ids.unique()),len(test_before_preprocess.product_ids.unique())

(266, 131)

In [20]:
len(set(train_before_preprocess.product_ids.unique())-set(test_before_preprocess.product_ids.unique()))

144

In [21]:
# test 아이템에 train 없는 아이템 확인
len(set(test_before_preprocess.product_ids.unique())-set(train_before_preprocess.product_ids.unique()))

9

In [22]:
# test 만 있는 item 제거
only_test_items = set(test_before_preprocess.product_ids.unique())-set(train_before_preprocess.product_ids.unique())
if_prepro_test = test_before_preprocess[~test_before_preprocess['product_ids'].isin(only_test_items)]

In [23]:
# test = test_before_preprocess.copy()
test = if_prepro_test.copy()

In [24]:
# train 변수 명 변경
train = train_before_preprocess.copy()

In [25]:
# test 전처리 진행했을 경우
print('원본 test 수:', len(test))
print('전처리 진행했을 경우 test 수:', len(if_prepro_test))

원본 test 수: 389
전처리 진행했을 경우 test 수: 389


# train test eda

### 전처리 전후 비교

In [26]:
print('train 전처리 전:',len(train_before_preprocess), 'train 전처리 후:',len(train))

train 전처리 전: 7351 train 전처리 후: 7351


In [27]:
print('test 전처리 전:',len(test_before_preprocess), 'test 전처리 후:',len(test))

test 전처리 전: 977 test 전처리 후: 389


### user 수 비교 

In [28]:
print('train 유저 수:',len(train.customer_id.unique()))

train 유저 수: 2931


In [29]:
print('test 유저 수:',len(test.customer_id.unique()))

test 유저 수: 258


In [30]:
# 신규 유저는 MP 같은 다른 방법으로 추천 진행해야 함
print('test 만 있는 신규 유저 :',len(set(test['customer_id'].unique())- set(train['customer_id'].unique())))

test 만 있는 신규 유저 : 103


### item 개수 비교

In [31]:
print('train 아이템 수 :',len(set(train.product_ids)), 'test 아이템 수 :',len(set(test.product_ids)))

train 아이템 수 : 266 test 아이템 수 : 122


In [32]:
print('train 만 있는 아이템 수:',  len(set(train.product_ids)-set(test.product_ids)))

train 만 있는 아이템 수: 144


In [33]:
print('test 만 있는 아이템 수:', len(set(test.product_ids) - set(train.product_ids)))

test 만 있는 아이템 수: 0


# 3. sparse matrix 만들기

## ALS MF Matrix

In [34]:
PdIds = train.product_ids.unique()

PdIdToIndex = {}
indexToPdId = {}

colIdx = 0

for PdId in PdIds:
    PdIdToIndex[PdId] = colIdx
    indexToPdId[colIdx] = PdId
    colIdx += 1
    
userIds = train.customer_id.unique()

userIdToIndex = {}
indexToUserId = {}

rowIdx = 0

for userId in userIds:
    userIdToIndex[userId] = rowIdx
    indexToUserId[rowIdx] = userId
    rowIdx += 1

import scipy.sparse as sp

rows = []
cols = []
vals = []

for row in train.itertuples():
    rows.append(userIdToIndex[row.customer_id])
    cols.append(PdIdToIndex[row.product_ids])
    vals.append(1)

purchase_sparse = sp.csr_matrix((vals, (rows, cols)), shape=(rowIdx,colIdx))

matrix = purchase_sparse.todense()
matrix

matrix([[1, 1, 0, ..., 0, 0, 0],
        [1, 0, 1, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [1, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0]])

### Most_popular_matrix

In [35]:
most_popular = train.groupby(['product_ids','name_x']).count()['customer_id'].reset_index()

### Medistream_prediction_matrix
- 메디스트림 메디마켓에서 제공하는 정렬 추천 성능 비교를 위한 df 구현
- 인기도순, 최신순, 과거순, 높은 가격순, 낮은 가격순, 이름순 (총 6 가지)
- 각각 구현해보고 학습 모델 대비 성능 비교

In [36]:
medistream_prediction_df = train[['date_created','regular_price','sale_price','three_months','product_ids','name_x']]
medistream_prediction_preprop_df = medistream_prediction_df.drop_duplicates(subset=['product_ids'], ignore_index=True)
medistream_prediction_preprop_df['date_created'] = pd.to_datetime(medistream_prediction_preprop_df['date_created'])
# sale_prices가 0이면 regular_price 값으로 채워넣어야하는데 0이 없음(전처리 필요 무)

<ipython-input-36-4d4e36d189e6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  medistream_prediction_preprop_df['date_created'] = pd.to_datetime(medistream_prediction_preprop_df['date_created'])


# Sparsity 확인

In [37]:
# Sparsity: 얼마나 비어있나?
matrix_size = purchase_sparse.shape[0]* purchase_sparse.shape[1]
num_purchases = len(purchase_sparse.nonzero()[0])
sparsity = 100 * (1 - (num_purchases / matrix_size))
sparsity

99.05713618744917

# 4. Model

## Model 학습 진행
- real test 만들기
- implict 라이브러리 사용(MF,LMF)
- MF 구현 모델 사용

In [38]:
# real test 
ground_trues = []
for user_id in test['customer_id'].unique():
    ground_trues.append({'id': user_id,\
    'items':list(test[test['customer_id']==user_id].product_ids)
    })

## ALS fit

In [39]:
als_model = ALS(factors=20, regularization=0.01, iterations = 50, random_state=42)
als_model.fit(purchase_sparse)

  0%|          | 0/50 [00:00<?, ?it/s]

In [40]:
# item, user vector 추출
als_item_factors = als_model.item_factors
als_user_factors = als_model.user_factors

In [41]:
# 각 shape 확인
als_item_factors.shape, als_user_factors.shape

((266, 20), (2931, 20))

## LMF fit

In [42]:
from implicit.lmf import LogisticMatrixFactorization as LMF

In [43]:
lmf_model = LMF(factors=20, regularization=0.001, iterations = 20, random_state=42)
lmf_model.fit(purchase_sparse)

  0%|          | 0/20 [00:00<?, ?it/s]

In [44]:
lmf_item_factors = lmf_model.item_factors
lmf_user_factors = lmf_model.user_factors

In [45]:
lmf_user_factors.shape, lmf_item_factors.shape

((2931, 22), (266, 22))

# 5. prediction

# ALS mf prediction

In [46]:
# 신규 유저인 경우 mp로 넣기
# 전체 도서에 대한 판매 만큼 정렬 후 넣기
most_popular_list = most_popular.sort_values(by='customer_id',ascending=False).index

# test 예측값, 이미 구매 했을 경우 제외
als_predict_list = []
for user_id in test['customer_id'].unique():
    try:
        result = als_model.recommend(userIdToIndex[user_id], purchase_sparse[userIdToIndex[user_id]], N=100)
        als_predict_list.append({'id':user_id ,'items':[indexToPdId[num] for num in result[0]]})
    except:
        train_purchase_list = list(train[train['customer_id']==user_id].product_ids)
        als_predict_list.append({'id':user_id ,'items':[most_popular.product_ids.loc[num] for num in most_popular_list \
                                                            if most_popular.product_ids.loc[num] not in train_purchase_list \
                                                            ]})
        
# 100 개만 예측하기
for idx, pred_list in enumerate(als_predict_list):
    als_predict_list[idx]['items'] = pred_list['items'][:100]

# ALS mf & latest prediction

In [47]:
# 신규 유저인 경우 mp로 넣기
# 전체 도서에 대한 판매 만큼 정렬 후 넣기
medistream_latest_list = medistream_prediction_preprop_df.sort_values(by='date_created', ascending=False).index

# test 예측값, 이미 구매 했을 경우 제외
als_latest_predict_list = []
for user_id in test['customer_id'].unique():
    try:
        result = als_model.recommend(userIdToIndex[user_id], purchase_sparse[userIdToIndex[user_id]], N=100)
        als_latest_predict_list.append({'id':user_id ,'items':[indexToPdId[num] for num in result[0]]})
    except:
        train_purchase_list = list(train[train['customer_id']==user_id].product_ids)
        als_latest_predict_list.append({'id':user_id ,'items':[medistream_prediction_preprop_df.product_ids.loc[num] for num in medistream_latest_list \
                                                            if medistream_prediction_preprop_df.product_ids.loc[num] not in train_purchase_list \
                                                            ]})
        
# 100 개만 예측하기
for idx, pred_list in enumerate(als_latest_predict_list):
    als_latest_predict_list[idx]['items'] = pred_list['items'][:100]

# LMF prediction

In [48]:
# 신규 유저 mp로 넣기
most_popular_list = most_popular.sort_values(by='customer_id',ascending=False).index

# test 예측값
lmf_predict_list = []
for user_id in test['customer_id'].unique():
    try:
        result = lmf_model.recommend(userIdToIndex[user_id], purchase_sparse[userIdToIndex[user_id]], N=100)
        lmf_predict_list.append({'id':user_id ,'items':[indexToPdId[num] for num in result[0]]})
    except:
        train_purchase_list = list(train[train['customer_id']==user_id].product_ids)
        lmf_predict_list.append({'id':user_id ,'items':[most_popular.product_ids.loc[num] for num in most_popular_list \
                                                            if most_popular.product_ids.loc[num] not in train_purchase_list \
                                                            ]})
        
# 100 개만 예측하기
for idx, pred_list in enumerate(lmf_predict_list):
    lmf_predict_list[idx]['items'] = pred_list['items'][:100]

# most popular prediction

In [49]:
# 전체 도서에 대한 판매 만큼 정렬 후 넣기
most_popular_list = most_popular.sort_values(by='customer_id',ascending=False).index

# test 예측값, 이미 구매 했을 경우 제외
predict_popular_list = []
for user_id in test['customer_id'].unique():
    train_purchase_list = list(train[train['customer_id']==user_id].product_ids)
    predict_popular_list.append({'id':user_id ,'items':[most_popular.product_ids.loc[num] for num in most_popular_list \
                                                            if most_popular.product_ids.loc[num] not in train_purchase_list \
                                                            ]})

# 100 개만 예측하기
for idx, pred_list in enumerate(predict_popular_list):
    predict_popular_list[idx]['items'] = pred_list['items'][:100]

# medistream prediction
- 메디스트림 메디마켓에서 제공하는 정렬 추천 성능 비교
- 인기도순, 최신순, 과거순, 높은 가격순, 낮은 가격순, 이름순 (총 6 가지)
- 각각 구현해보고 학습 모델 대비 성능 비교

In [50]:
# 인기도순
medistream_popular_list = medistream_prediction_preprop_df.sort_values(by='three_months', ascending=False).index
# 최신순
medistream_latest_list = medistream_prediction_preprop_df.sort_values(by='date_created', ascending=False).index
# 오랜된 순
medistream_oldest_list = medistream_prediction_preprop_df.sort_values(by='date_created', ascending=True).index
# 높은 가격 순
medistream_high_price_list = medistream_prediction_preprop_df.sort_values(by='sale_price', ascending=False).index
# 낮은 가격 순
medistream_low_price_list = medistream_prediction_preprop_df.sort_values(by='sale_price', ascending=True).index
# 이름 순
medistream_name_sort_list = medistream_prediction_preprop_df.sort_values(by='name_x',ascending=True).index

def medistream_prediction_method(predict_num:int ,medi_predict_list:list)->list:
    medistream_predict_list = []
    for user_id in test['customer_id'].unique():
        medistream_predict_list.append({'id':user_id ,'items':[medistream_prediction_preprop_df.product_ids.loc[num] \
                                                                       for num in medi_predict_list]})

    # 100 개만 예측하기
    for idx, pred_list in enumerate(medistream_predict_list):
        medistream_predict_list[idx]['items'] = pred_list['items'][:predict_num]
        
    return medistream_predict_list

In [51]:
medistream_predict_popular_list = medistream_prediction_method(100, medistream_popular_list)
medistream_predict_latest_list = medistream_prediction_method(100, medistream_latest_list)
medistream_predict_oldest_list = medistream_prediction_method(100, medistream_oldest_list)
medistream_predict_high_price_list = medistream_prediction_method(100, medistream_high_price_list)
medistream_predict_low_price_list = medistream_prediction_method(100, medistream_low_price_list)
medistream_predict_name_sort_list = medistream_prediction_method(100, medistream_name_sort_list)

# 6. evaluation

## NDCG & Entropy Diversity 평가지표

In [52]:
class CustomEvaluator:
    # relavence 모두 1로 동일하게 봄
    def _idcg(self, l):
        return sum((1.0 / np.log(i + 2) for i in range(l)))
    

    def __init__(self):
        self._idcgs = [self._idcg(i) for i in range(1000)]
    '''
    idcgs 예시, item 3개 추천되므로 3.074281787960283 가 됩니다.
    [0, 1.4426950408889634, 2.352934267515801, 3.074281787960283]
    '''

    def _ndcg(self, gt, rec):
        dcg = 0.0
        for i, r in enumerate(rec):
            if r in gt:
                dcg += 1.0 / np.log(i + 2)

        return dcg / self._idcgs[len(gt)]
    
    def _entropy_diversity(self,rec_list):
        import six
        import math
        
        topn = len(rec_list[0]['items'])
        users = [i.get('id',None) for i in rec_list]
        sz = float(len(users)) * topn
        freq = {}
        for rec in rec_list:
            for r in rec['items']:
                freq[r] = freq.get(r, 0) + 1
        ent = -sum([v / sz * math.log(v / sz) for v in six.itervalues(freq)])
        return ent

    def _eval(self, gt_list, rec_list):
        gt_dict = {g["id"]: g for g in gt_list}
        ndcg_score = 0.0

        for rec in rec_list:
            gt = gt_dict[rec["id"]]
            ndcg_score += self._ndcg(gt["items"], rec["items"])


        ndcg_score = ndcg_score / len(rec_list)
        ent = self._entropy_diversity(rec_list)
        
        return ndcg_score, ent

    def evaluate(self, gt_list, rec_list):
        try:
            ndcg_score, ent_score = self._eval(gt_list, rec_list)
            print(f"NDCG: {ndcg_score:.6}")
            print(f"Entropy Diversity: {ent_score:.6} ")
        except Exception as e:
            print(e)


# ALS NDCG

In [53]:
# ALS 
evaluator = CustomEvaluator()
evaluator.evaluate(ground_trues, als_predict_list)

NDCG: 0.191105
Entropy Diversity: 5.26323 


In [54]:
len(als_predict_list),len(ground_trues)

(258, 258)

In [55]:
# 아이템 맞춘 개수
cnt = 0
for gt, pred_list in zip(ground_trues, als_predict_list):
    for pred in pred_list['items']:
        if pred in gt['items']:
            cnt += 1
cnt

238

# ALS & latest NDCG

In [56]:
# ALS 
evaluator = CustomEvaluator()
evaluator.evaluate(ground_trues, als_latest_predict_list)

NDCG: 0.127086
Entropy Diversity: 5.32488 


In [57]:
len(als_latest_predict_list),len(ground_trues)

(258, 258)

In [58]:
# 아이템 맞춘 개수
cnt = 0
for gt, pred_list in zip(ground_trues, als_latest_predict_list):
    for pred in pred_list['items']:
        if pred in gt['items']:
            cnt += 1
cnt

202

# LMF NDCG

In [59]:
# ALS 
evaluator = CustomEvaluator()
evaluator.evaluate(ground_trues, lmf_predict_list)

NDCG: 0.226276
Entropy Diversity: 5.05531 


In [60]:
len(lmf_predict_list),len(ground_trues)

(258, 258)

In [61]:
# 아이템 맞춘 개수
cnt = 0
for gt, pred_list in zip(ground_trues, lmf_predict_list):
    for pred in pred_list['items']:
        if pred in gt['items']:
            cnt += 1
cnt

266

# most popular NDCG

In [62]:
# most popular
evaluator = CustomEvaluator()
evaluator.evaluate(ground_trues, predict_popular_list)

NDCG: 0.247185
Entropy Diversity: 4.66782 


In [63]:
len(predict_popular_list),len(ground_trues)

(258, 258)

In [64]:
# 아이템 맞춘 개수
cnt = 0
for gt, pred_list in zip(ground_trues, predict_popular_list):
    for pred in pred_list['items']:
        if pred in gt['items']:
            cnt += 1
cnt

278

## medistream prediction NDCG & Entropy

In [65]:
def medistream_prediction(ground_trues:list, predict_list:list):
    evaluator = CustomEvaluator()
    ndcg, entropy = evaluator._eval(ground_trues, predict_list)
    
    assert len(predict_list) == len(ground_trues)
    
    cnt = 0
    for gt, pred_list in zip(ground_trues, predict_list):
        for pred in pred_list['items']:
            if pred in gt['items']:
                cnt += 1
    return ndcg, entropy, cnt

In [66]:
medistream_predict_score = {'medistream_predict':['medi_popular','latest','oldest','high_price','low_price','name_sort'], \
                            'ndcg':[], 'entropy':[], 'cnt':[]}

medistream_predict_list = [medistream_predict_popular_list, medistream_predict_latest_list, medistream_predict_oldest_list,\
                          medistream_predict_high_price_list, medistream_predict_low_price_list, medistream_predict_name_sort_list]

for medistream_predict in medistream_predict_list:
    ndcg, entropy, cnt = medistream_prediction(ground_trues, medistream_predict)
    medistream_predict_score['ndcg'].append(ndcg)
    medistream_predict_score['entropy'].append(entropy)
    medistream_predict_score['cnt'].append(cnt)
pd.DataFrame(medistream_predict_score)    

,medistream_predict,ndcg,entropy,cnt
0,medi_popular,0.292157,4.60517,340
1,latest,0.117948,4.60517,186
2,oldest,0.117723,4.60517,161
3,high_price,0.072215,4.60517,135
4,low_price,0.065859,4.60517,137
5,name_sort,0.066386,4.60517,107


- 최신순, 인기도 순으로 점수가 높게 나왔습니다.

# 7. hyper parameter tuning

## 7-1. ALS MF hypter parameter tuning

In [67]:
# def model_parameter_tuning(model:str, side_rec_list:pd.DataFrame())->pd.DataFrame():
#     hyper_parameter = {'factor':[],'regularization':[],'iteration':[],'NDCG':[],'entropy':[]}

#     factors = [5,10,15,20]
#     regularizations = [0.01,0.005]
#     iterations = [5,10,15,20,25,30,40,50]

#     for factor in factors:
#         for regularization in regularizations:
#             for iteration in iterations:
#                 if model == 'als':
#                     als_model = ALS(factors=factor, regularization=regularization, iterations = iteration, random_state=42)
#                     als_model.fit(purchase_sparse, show_progress=False)
                    
#                     # 신규 유저인 경우 mp로 넣기
#                     # 전체 도서에 대한 판매 만큼 정렬 후 넣기
#                     side_rec_list_index = side_rec_list.sort_values(by='customer_id',ascending=False).index

#                     # test 예측값, 이미 구매 했을 경우 제외
#                     predict_list = []
#                     for user_id in test['customer_id'].unique():
#                         try:
#                             result = als_model.recommend(userIdToIndex[user_id], purchase_sparse[userIdToIndex[user_id]], N=100)
#                             predict_list.append({'id':user_id ,'items':[indexToPdId[num] for num in result[0]]})
#                         except:
#                             train_purchase_list = list(train[train['customer_id']==user_id].product_ids)
#                             predict_list.append({'id':user_id ,'items':[side_rec_list.product_ids.loc[num] for num in side_rec_list_index \
#                                                                                 if side_rec_list.product_ids.loc[num] not in train_purchase_list \
#                                                                                 ]})
#                 elif model == 'lmf':
#                     lmf_model = LMF(factors=factor, regularization=regularization, iterations = iteration, random_state=42)
#                     lmf_model.fit(purchase_sparse, show_progress=False)
                    
#                     # 신규 유저인 경우 mp로 넣기
#                     # 전체 도서에 대한 판매 만큼 정렬 후 넣기
#                     side_rec_list_index = side_rec_list.sort_values(by='customer_id',ascending=False).index

#                     # test 예측값, 이미 구매 했을 경우 제외
#                     predict_list = []
#                     for user_id in test['customer_id'].unique():
#                         try:
#                             result = lmf_model.recommend(userIdToIndex[user_id], purchase_sparse[userIdToIndex[user_id]], N=100)
#                             predict_list.append({'id':user_id ,'items':[indexToPdId[num] for num in result[0]]})
#                         except:
#                             train_purchase_list = list(train[train['customer_id']==user_id].product_ids)
#                             predict_list.append({'id':user_id ,'items':[side_rec_list.product_ids.loc[num] for num in side_rec_list_index \
#                                                                                 if side_rec_list.product_ids.loc[num] not in train_purchase_list \
#                                                                                 ]})
                    


#                 # 100 개만 예측하기
#                 for idx, pred_list in enumerate(predict_list):
#                     predict_list[idx]['items'] = pred_list['items'][:100]

#                 # ALS 
#                 evaluator = CustomEvaluator()
#                 ndcg,entropy = evaluator._eval(ground_trues, predict_list)

#                 hyper_parameter['factor'].append(factor)
#                 hyper_parameter['regularization'].append(regularization)
#                 hyper_parameter['iteration'].append(iteration)
#                 hyper_parameter['NDCG'].append(ndcg)
#                 hyper_parameter['entropy'].append(entropy)
#     return hyper_parameter

In [68]:
# model_parameter_tuning('als',most_popular)

In [69]:
als_mf_hyper_parameter = {'factor':[],'regularization':[],'iteration':[],'NDCG':[],'entropy':[]}

factors = [5,10,15,20]
regularizations = [0.01,0.005]
iterations = [5,10,15,20,25,30,40,50]

for factor in factors:
    for regularization in regularizations:
        for iteration in iterations:
            als_model = ALS(factors=factor, regularization=regularization, iterations = iteration, random_state=42)
            als_model.fit(purchase_sparse, show_progress=False)

            # 신규 유저인 경우 mp로 넣기
            # 전체 도서에 대한 판매 만큼 정렬 후 넣기
            most_popular_list = most_popular.sort_values(by='customer_id',ascending=False).index

            # test 예측값, 이미 구매 했을 경우 제외
            als_predict_list = []
            for user_id in test['customer_id'].unique():
                try:
                    result = als_model.recommend(userIdToIndex[user_id], purchase_sparse[userIdToIndex[user_id]], N=100)
                    als_predict_list.append({'id':user_id ,'items':[indexToPdId[num] for num in result[0]]})
                except:
                    train_purchase_list = list(train[train['customer_id']==user_id].product_ids)
                    als_predict_list.append({'id':user_id ,'items':[most_popular.product_ids.loc[num] for num in most_popular_list \
                                                                        if most_popular.product_ids.loc[num] not in train_purchase_list \
                                                                        ]})

            # 100 개만 예측하기
            for idx, pred_list in enumerate(als_predict_list):
                als_predict_list[idx]['items'] = pred_list['items'][:100]

            # ALS 
            evaluator = CustomEvaluator()
            ndcg, entropy = evaluator._eval(ground_trues, als_predict_list)

            als_mf_hyper_parameter['factor'].append(factor)
            als_mf_hyper_parameter['regularization'].append(regularization)
            als_mf_hyper_parameter['iteration'].append(iteration)
            als_mf_hyper_parameter['NDCG'].append(ndcg)
            als_mf_hyper_parameter['entropy'].append(entropy)

In [70]:
pd.DataFrame(als_mf_hyper_parameter).sort_values(by='NDCG',ascending=False).head()

,factor,regularization,iteration,NDCG,entropy
31,10,0.005,50,0.213695,5.113279
23,10,0.010,50,0.213687,5.113402
0,5,0.010,5,0.213567,4.913463
8,5,0.005,5,0.213393,4.913274
26,10,0.005,15,0.211325,5.123055


## 7-2. LMF hypter parameter tuning

In [71]:
lmf_hyper_parameter = {'factor':[],'regularization':[],'iteration':[],'NDCG':[],'entropy':[]}

factors = [5,10,15,20]
regularizations = [0.01,0.005]
iterations = [5,10,15,20,25,30,40,50]

for factor in factors:
    for regularization in regularizations:
        for iteration in iterations:
            lmf_model = LMF(factors=factor, regularization=regularization, iterations = iteration, random_state=42)
            lmf_model.fit(purchase_sparse, show_progress=False)
            
            # 신규 유저 mp로 넣기
            most_popular_list = most_popular.sort_values(by='customer_id',ascending=False).index

            # test 예측값
            lmf_predict_list = []
            for user_id in test['customer_id'].unique():
                try:
                    result = lmf_model.recommend(userIdToIndex[user_id], purchase_sparse[userIdToIndex[user_id]], N=100)
                    lmf_predict_list.append({'id':user_id ,'items':[indexToPdId[num] for num in result[0]]})
                except:
                    train_purchase_list = list(train[train['customer_id']==user_id].product_ids)
                    lmf_predict_list.append({'id':user_id ,'items':[most_popular.product_ids.loc[num] for num in most_popular_list \
                                                                        if most_popular.product_ids.loc[num] not in train_purchase_list \
                                                                        ]})

            # 100 개만 예측하기
            for idx, pred_list in enumerate(lmf_predict_list):
                lmf_predict_list[idx]['items'] = pred_list['items'][:100]
                
            # LMF
            evaluator = CustomEvaluator()
            ndcg, entropy = evaluator._eval(ground_trues, lmf_predict_list)
            
            lmf_hyper_parameter['factor'].append(factor)
            lmf_hyper_parameter['regularization'].append(regularization)
            lmf_hyper_parameter['iteration'].append(iteration)
            lmf_hyper_parameter['NDCG'].append(ndcg)
            lmf_hyper_parameter['entropy'].append(entropy)

In [72]:
pd.DataFrame(lmf_hyper_parameter).sort_values(by='NDCG',ascending=False).head()

,factor,regularization,iteration,NDCG,entropy
55,20,0.010,50,0.237090,4.972683
63,20,0.005,50,0.234154,4.973066
51,20,0.010,20,0.229583,5.053129
59,20,0.005,20,0.226710,5.057017
54,20,0.010,40,0.226504,4.983001


# 8. 결론

- als mf : 0.283429 (factor: 10, regularization: 0.005, iteration: 15)
- lmf : 0.292581    (factor: 15, regularization: 0.005, iteration: 25)
- mp : 0.262317
- medi_popular : 0.398149
- latest : 0.574001

최근 아이템으로 정렬해주었을 때 가장 높은 점수가 나왔음..

In [73]:
all_prediction_df = {'first_day':[],'last_day':[],'train_데이터수':[],'train_유저수':[],'test_데이터수':[],\
    'test_유저수':[],'test_신규유저수':[],'test_신규아이템수':[],'원본_test수':[],'전처리진행test수':[],\
    'als_mf':[],'lmf':[],'mp':[],'medi_popular':[],'latest':[],\
    'oldest':[],'high_price':[],'low_price':[],'name_sort':[],\
     'als_mf_entropy':[],'lmf_entropy':[],'mp_entropy':[],'medi_popular_entropy':[],'latest_entropy':[],\
     'oldest_entropy':[],'high_price_entropy':[],'low_price_entropy':[],'name_sort_entropy':[]}
medistream_predict_df = pd.DataFrame(medistream_predict_score)

all_prediction_df['first_day'].append(str(datetime.date(train['date_paid'].min()))+' '+str(datetime.date(train['date_paid'].max())))
all_prediction_df['last_day'].append(str(datetime.date(test['date_paid'].min()))+' '+str(datetime.date(test['date_paid'].max())))
all_prediction_df['train_데이터수'].append(len(train))
all_prediction_df['train_유저수'].append(len(set(train.customer_id)))
all_prediction_df['test_데이터수'].append(len(test))
all_prediction_df['test_유저수'].append(len(set(test.customer_id)))
all_prediction_df['test_신규유저수'].append(len(set(test['customer_id'].unique())- set(train['customer_id'].unique())))
all_prediction_df['test_신규아이템수'].append(len(set(test_before_preprocess.product_ids.unique())-set(train_before_preprocess.product_ids.unique())))
all_prediction_df['원본_test수'].append(len(test))
all_prediction_df['전처리진행test수'].append(len(if_prepro_test))

# ndcg
all_prediction_df['als_mf'].append(pd.DataFrame(als_mf_hyper_parameter).sort_values(by='NDCG',ascending=False)['NDCG'].iloc[0])
all_prediction_df['lmf'].append(pd.DataFrame(lmf_hyper_parameter).sort_values(by='NDCG',ascending=False)['NDCG'].iloc[0])
all_prediction_df['mp'].append(evaluator._eval(ground_trues, predict_popular_list)[0])
all_prediction_df['medi_popular'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='medi_popular'].iloc[0]['ndcg'])
all_prediction_df['latest'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='latest'].iloc[0]['ndcg'])
all_prediction_df['oldest'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='oldest'].iloc[0]['ndcg'])
all_prediction_df['high_price'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='high_price'].iloc[0]['ndcg'])
all_prediction_df['low_price'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='low_price'].iloc[0]['ndcg'])
all_prediction_df['name_sort'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='name_sort'].iloc[0]['ndcg'])

# entropy
all_prediction_df['als_mf_entropy'].append(pd.DataFrame(als_mf_hyper_parameter).sort_values(by='entropy',ascending=False)['entropy'].iloc[0])
all_prediction_df['lmf_entropy'].append(pd.DataFrame(lmf_hyper_parameter).sort_values(by='entropy',ascending=False)['entropy'].iloc[0])
all_prediction_df['mp_entropy'].append(evaluator._eval(ground_trues, predict_popular_list)[0])
all_prediction_df['medi_popular_entropy'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='medi_popular'].iloc[0]['entropy'])
all_prediction_df['latest_entropy'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='latest'].iloc[0]['entropy'])
all_prediction_df['oldest_entropy'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='oldest'].iloc[0]['entropy'])
all_prediction_df['high_price_entropy'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='high_price'].iloc[0]['entropy'])
all_prediction_df['low_price_entropy'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='low_price'].iloc[0]['entropy'])
all_prediction_df['name_sort_entropy'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='name_sort'].iloc[0]['entropy'])

print('train 총 기간:',train['date_paid'].max()-train['date_paid'].min())
print('test 총 기간:',test['date_paid'].max()-test['date_paid'].min())
display(pd.DataFrame(all_prediction_df))

train 총 기간: 162 days 13:23:20.039000
test 총 기간: 21 days 03:48:41.346000


,first_day,last_day,train_데이터수,train_유저수,test_데이터수,test_유저수,test_신규유저수,test_신규아이템수,원본_test수,전처리진행test수,als_mf,lmf,mp,medi_popular,latest,oldest,high_price,low_price,name_sort,als_mf_entropy,lmf_entropy,mp_entropy,medi_popular_entropy,latest_entropy,oldest_entropy,high_price_entropy,low_price_entropy,name_sort_entropy
0,2022-03-13 2022-08-22,2022-08-23 2022-09-13,7351,2931,389,258,103,9,389,389,0.213695,0.23709,0.247185,0.292157,0.117948,0.117723,0.072215,0.065859,0.066386,5.264037,5.236191,0.247185,4.60517,4.60517,4.60517,4.60517,4.60517,4.60517


# 9. 추천된 items 확인

In [74]:
products_df = pd.read_json("/fastcampus-data/products/products.json")
products_df = key_to_element(['_id'],products_df)

100%|██████████| 5141/5141 [00:00<00:00, 695398.51it/s]


In [75]:
# pred_item, rea_item 비교
def pred_real_dataframe(user_num):
    pred_items_names = []
    predict_dict = als_predict_list[user_num]['items']
    for item in predict_dict:
        pred_items_names.append(products_df[products_df['_id'] == item].meta_title.unique())

    real_items_names = []
    trues_dict = ground_trues[user_num]['items']
    for item in trues_dict:
        real_items_names.append(products_df[products_df['_id'] == item].meta_title.unique())
    return pd.DataFrame({'pred_item':pred_items_names,'real_item':real_items_names})
    

In [76]:
# train_item, pred_item, real_item 비교
def train_pred_items(user_nums):
    train_pred_items_df = pd.DataFrame(columns=['train_item','pred_item'])
    for user_num in range(1,user_nums):
        train_item_names = []
        for idx in grouped_purchased[grouped_purchased['customer_id']==ground_trues[user_num]['id']].product_ids:
            train_item_names.append(products_df[products_df['_id'] == idx].meta_title.unique()[0])

        pred_items_names = []
        predict_dict = als_predict_list[user_num]['items']
        for item in predict_dict:
            pred_items_names.append(products_df[products_df['_id'] == item].meta_title.unique())


        
        train_pred_items_df.loc[user_num,'train_item'] = train_item_names
        train_pred_items_df.loc[user_num,'pred_item'] = pred_items_names
    return train_pred_items_df


In [77]:
train_pred_items(13)

NameError: name 'grouped_purchased' is not defined

In [ ]:
# 예측 유저 구매 횟수 확인
pd.DataFrame(purchase_sparse[1].todense()).T.value_counts()

In [ ]:
print('첫날 :',train['date_paid'].min(),'마지막 날:',train['date_paid'].max())
print('train 총 기간:',train['date_paid'].max()-train['date_paid'].min())
print('______________________________________________________')
print('첫날 :',test['date_paid'].min(),'마지막 날:',test['date_paid'].max())
print('test 총 기간:',test['date_paid'].max()-test['date_paid'].min())
print('______________________________________________________')
print('train 데이터수:', len(train))
print('______________________________________________________')
print('train 유저수:',len(set(train.customer_id)))
print('______________________________________________________')
print('test 데이터수:',len(test))
print('______________________________________________________')
print('test 유저수:',len(set(test.customer_id)))
print('______________________________________________________')
print('test 신규 유저 수:',len(set(test['customer_id'].unique())- set(train['customer_id'].unique())))
print('______________________________________________________')
print('test 신규 아이템 수:',len(set(test_before_preprocess.product_ids.unique())-set(train_before_preprocess.product_ids.unique())))
# test 전처리 진행했을 경우
print('______________________________________________________')
print('원본 test 수:', len(test))
print('______________________________________________________')
print('전처리 진행했을 경우 test 수:', len(if_prepro_test))
print('______________________________________________________')
print('mf')
display(pd.DataFrame(als_mf_hyper_parameter).sort_values(by='NDCG',ascending=False).head(2))
print('______________________________________________________')
print('lmf')
display(pd.DataFrame(lmf_hyper_parameter).sort_values(by='NDCG',ascending=False).head(2))
print('______________________________________________________')
print('mp')
evaluator.evaluate(ground_trues, predict_popular_list)
print('______________________________________________________')
display(pd.DataFrame(medistream_predict_score))